# 从零实现神经元

## 📚 代码示例关联

**本示例位置**：这是神经网络基础的第二个代码示例

**基于**：
- [从零实现感知器](../01_从零实现感知器.ipynb) - 神经元是感知器的扩展，添加了激活函数

**扩展为**：
- [从零实现前馈神经网络](../../02_前馈神经网络/代码示例/01_从零实现前馈神经网络.ipynb) - 前馈神经网络由多个神经元组成
- [PyTorch基础入门](../../../04_PyTorch_TensorFlow/01_PyTorch基础/代码示例/01_PyTorch基础入门.ipynb) - 使用框架实现神经元

**相关示例**：
- [理解激活函数](../../练习题/基础练习/练习1_理解激活函数.ipynb) - 基础练习
- [完整神经元训练流程](../../Jupyter练习/01_完整神经元训练流程.ipynb) - 完整训练流程

---

## 学习目标
- 理解神经元的基本结构和工作原理
- 掌握不同类型的激活函数（Sigmoid、Tanh、ReLU等）
- 能够从零实现带不同激活函数的神经元
- 理解激活函数对输出的影响
- 可视化激活函数及其导数

## 课程概述
本notebook将带你从零开始实现神经元，包括：
1. 激活函数的实现（Sigmoid、Tanh、ReLU等）
2. 神经元类的实现
3. 激活函数的可视化
4. 不同激活函数的对比分析


In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# 设置随机种子，确保结果可复现
np.random.seed(42)

print("环境准备完成！")


## 1. 激活函数实现

我们将实现常用的激活函数，包括函数本身和其导数。

### 1.1 激活函数类定义


In [ ]:
class ActivationFunctions:
    """
    激活函数集合
    包含常用激活函数及其导数
    """
    
    @staticmethod
    def sigmoid(z):
        """
        Sigmoid激活函数
        公式: σ(z) = 1 / (1 + e^(-z))
        输出范围: [0, 1]
        """
        z = np.clip(z, -500, 500)  # 防止溢出
        return 1 / (1 + np.exp(-z))
    
    @staticmethod
    def sigmoid_derivative(z):
        """Sigmoid函数的导数: σ'(z) = σ(z) * (1 - σ(z))"""
        s = ActivationFunctions.sigmoid(z)
        return s * (1 - s)
    
    @staticmethod
    def tanh(z):
        """Tanh激活函数，输出范围: [-1, 1]"""
        return np.tanh(z)
    
    @staticmethod
    def tanh_derivative(z):
        """Tanh函数的导数: tanh'(z) = 1 - tanh^2(z)"""
        return 1 - np.tanh(z) ** 2
    
    @staticmethod
    def relu(z):
        """ReLU激活函数: ReLU(z) = max(0, z)"""
        return np.maximum(0, z)
    
    @staticmethod
    def relu_derivative(z):
        """ReLU函数的导数: ReLU'(z) = 1 if z > 0 else 0"""
        return (z > 0).astype(float)
    
    @staticmethod
    def leaky_relu(z, alpha=0.01):
        """Leaky ReLU激活函数: LeakyReLU(z) = max(αz, z)"""
        return np.maximum(alpha * z, z)
    
    @staticmethod
    def leaky_relu_derivative(z, alpha=0.01):
        """Leaky ReLU函数的导数"""
        return np.where(z > 0, 1.0, alpha)

print("激活函数定义完成！")


## 2. 可视化激活函数

绘制不同激活函数及其导数的图像，理解它们的特性。

### 2.1 生成输入值和计算激活函数


## 4. 测试不同激活函数的神经元

使用相同的数据和训练过程，测试不同激活函数的神经元。


In [ ]:
# 生成简单的训练数据
np.random.seed(42)
n_samples = 100

# 正类样本（标签=1）
X1 = np.random.randn(n_samples // 2, 2) + [1, 1]
y1 = np.ones(n_samples // 2)

# 负类样本（标签=0）
X0 = np.random.randn(n_samples // 2, 2) + [-1, -1]
y0 = np.zeros(n_samples // 2)

# 合并数据
X = np.vstack([X1, X0])
y = np.hstack([y1, y0])

print("=" * 60)
print("数据信息")
print("=" * 60)
print(f"总样本数: {len(X)}")
print(f"特征数: {X.shape[1]}")
print(f"正类样本数: {np.sum(y == 1)}")
print(f"负类样本数: {np.sum(y == 0)}")


In [ ]:
# 训练不同激活函数的神经元
activations = ['sigmoid', 'tanh', 'relu', 'leaky_relu']
neurons = {}
losses_history = {}

learning_rate = 0.1
n_epochs = 100

for activation in activations:
    print(f"\n{'='*60}")
    print(f"训练 {activation.upper()} 神经元")
    print(f"{'='*60}")
    
    # 创建神经元
    neuron = Neuron(n_features=2, activation=activation)
    neurons[activation] = neuron
    
    # 记录损失历史
    losses = []
    
    # 训练循环
    for epoch in range(n_epochs):
        # 前向传播
        output, z = neuron.forward(X)
        
        # 计算损失（均方误差）
        loss = np.mean((output - y) ** 2)
        losses.append(loss)
        
        # 计算梯度
        grad_output = 2 * (output - y) / len(y)
        
        # 反向传播
        grad_weights, grad_bias = neuron.backward(X, z, output, grad_output)
        
        # 更新参数
        neuron.update(grad_weights, grad_bias, learning_rate)
        
        # 每10个epoch打印一次
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}/{n_epochs}, Loss: {loss:.4f}")
    
    losses_history[activation] = losses
    
    # 计算最终准确率
    final_output, _ = neuron.forward(X)
    predictions = (final_output > 0.5).astype(int)
    accuracy = np.mean(predictions == y)
    print(f"\n最终准确率: {accuracy:.4f} ({accuracy*100:.2f}%)")


In [ ]:
# 生成简单的训练数据
np.random.seed(42)
n_samples = 100

# 正类样本（标签=1）
X1 = np.random.randn(n_samples // 2, 2) + [1, 1]
y1 = np.ones(n_samples // 2)

# 负类样本（标签=0）
X0 = np.random.randn(n_samples // 2, 2) + [-1, -1]
y0 = np.zeros(n_samples // 2)

# 合并数据
X = np.vstack([X1, X0])
y = np.hstack([y1, y0])

print("=" * 60)
print("数据信息")
print("=" * 60)
print(f"总样本数: {len(X)}")
print(f"特征数: {X.shape[1]}")
print(f"正类样本数: {np.sum(y == 1)}")
print(f"负类样本数: {np.sum(y == 0)}")


## 5. 可视化训练过程

对比不同激活函数的训练损失曲线。


In [ ]:
# 训练不同激活函数的神经元
activations = ['sigmoid', 'tanh', 'relu', 'leaky_relu']
neurons = {}
losses_history = {}

learning_rate = 0.1
n_epochs = 100

for activation in activations:
    print(f"\n{'='*60}")
    print(f"训练 {activation.upper()} 神经元")
    print(f"{'='*60}")
    
    # 创建神经元
    neuron = Neuron(n_features=2, activation=activation)
    neurons[activation] = neuron
    
    # 记录损失历史
    losses = []
    
    # 训练循环
    for epoch in range(n_epochs):
        # 前向传播
        output, z = neuron.forward(X)
        
        # 计算损失（均方误差）
        loss = np.mean((output - y) ** 2)
        losses.append(loss)
        
        # 计算梯度
        grad_output = 2 * (output - y) / len(y)
        
        # 反向传播
        grad_weights, grad_bias = neuron.backward(X, z, output, grad_output)
        
        # 更新参数
        neuron.update(grad_weights, grad_bias, learning_rate)
        
        # 每10个epoch打印一次
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}/{n_epochs}, Loss: {loss:.4f}")
    
    losses_history[activation] = losses
    
    # 计算最终准确率
    final_output, _ = neuron.forward(X)
    predictions = (final_output > 0.5).astype(int)
    accuracy = np.mean(predictions == y)
    print(f"\n最终准确率: {accuracy:.4f} ({accuracy*100:.2f}%)")


In [ ]:
# 绘制损失曲线
plt.figure(figsize=(12, 6))

for activation in activations:
    plt.plot(losses_history[activation], label=activation.upper(), linewidth=2)

plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss (MSE)', fontsize=12)
plt.title('不同激活函数的训练损失对比', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("=" * 60)
print("训练结果分析")
print("=" * 60)
print("1. Sigmoid和Tanh：适合二分类问题，输出范围有限")
print("2. ReLU和Leaky ReLU：可能输出较大值，需要调整损失函数或阈值")
print("3. 不同激活函数有不同的收敛速度和最终性能")


In [ ]:
# 生成输入值范围
z = np.linspace(-5, 5, 1000)

# 计算各种激活函数的值
sigmoid_z = ActivationFunctions.sigmoid(z)
tanh_z = ActivationFunctions.tanh(z)
relu_z = ActivationFunctions.relu(z)
leaky_relu_z = ActivationFunctions.leaky_relu(z)

# 计算导数
sigmoid_derivative_z = ActivationFunctions.sigmoid_derivative(z)
tanh_derivative_z = ActivationFunctions.tanh_derivative(z)
relu_derivative_z = ActivationFunctions.relu_derivative(z)
leaky_relu_derivative_z = ActivationFunctions.leaky_relu_derivative(z)

# 绘制激活函数
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 第一行：激活函数
axes[0, 0].plot(z, sigmoid_z, 'b-', linewidth=2, label='Sigmoid')
axes[0, 0].plot(z, tanh_z, 'r-', linewidth=2, label='Tanh')
axes[0, 0].set_xlabel('输入 z', fontsize=12)
axes[0, 0].set_ylabel('输出 f(z)', fontsize=12)
axes[0, 0].set_title('Sigmoid vs Tanh', fontsize=14)
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(z, relu_z, 'g-', linewidth=2, label='ReLU')
axes[0, 1].plot(z, leaky_relu_z, 'm-', linewidth=2, label='Leaky ReLU (α=0.01)')
axes[0, 1].set_xlabel('输入 z', fontsize=12)
axes[0, 1].set_ylabel('输出 f(z)', fontsize=12)
axes[0, 1].set_title('ReLU vs Leaky ReLU', fontsize=14)
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# 第二行：导数
axes[1, 0].plot(z, sigmoid_derivative_z, 'b-', linewidth=2, label="Sigmoid'")
axes[1, 0].plot(z, tanh_derivative_z, 'r-', linewidth=2, label="Tanh'")
axes[1, 0].set_xlabel('输入 z', fontsize=12)
axes[1, 0].set_ylabel("导数 f'(z)", fontsize=12)
axes[1, 0].set_title('Sigmoid vs Tanh 导数', fontsize=14)
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(z, relu_derivative_z, 'g-', linewidth=2, label="ReLU'")
axes[1, 1].plot(z, leaky_relu_derivative_z, 'm-', linewidth=2, label="Leaky ReLU'")
axes[1, 1].set_xlabel('输入 z', fontsize=12)
axes[1, 1].set_ylabel("导数 f'(z)", fontsize=12)
axes[1, 1].set_title('ReLU vs Leaky ReLU 导数', fontsize=14)
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("=" * 60)
print("激活函数特性总结")
print("=" * 60)
print("Sigmoid: 输出[0,1]，适合概率输出，但容易梯度消失")
print("Tanh: 输出[-1,1]，零中心化，梯度比Sigmoid大")
print("ReLU: 输出[0,+∞)，计算简单，解决梯度消失，最常用")
print("Leaky ReLU: ReLU的改进，解决Dead ReLU问题")


In [ ]:
class Neuron:
    """
    神经元类
    支持不同的激活函数
    """
    
    def __init__(self, n_features, activation='sigmoid'):
        """
        初始化神经元
        参数:
            n_features: 输入特征数
            activation: 激活函数类型（'sigmoid', 'tanh', 'relu', 'leaky_relu'）
        """
        self.n_features = n_features
        self.activation = activation
        
        # 初始化权重和偏置（使用Xavier或He初始化）
        if activation in ['sigmoid', 'tanh']:
            limit = np.sqrt(1.0 / n_features)
            self.weights = np.random.uniform(-limit, limit, n_features)
        else:  # ReLU, Leaky ReLU
            self.weights = np.random.randn(n_features) * np.sqrt(2.0 / n_features)
        
        self.bias = 0.0
        self._set_activation_functions()
    
    def _set_activation_functions(self):
        """设置激活函数和导数函数"""
        if self.activation == 'sigmoid':
            self.activation_func = ActivationFunctions.sigmoid
            self.activation_derivative = ActivationFunctions.sigmoid_derivative
        elif self.activation == 'tanh':
            self.activation_func = ActivationFunctions.tanh
            self.activation_derivative = ActivationFunctions.tanh_derivative
        elif self.activation == 'relu':
            self.activation_func = ActivationFunctions.relu
            self.activation_derivative = ActivationFunctions.relu_derivative
        elif self.activation == 'leaky_relu':
            self.activation_func = ActivationFunctions.leaky_relu
            self.activation_derivative = ActivationFunctions.leaky_relu_derivative
        else:
            raise ValueError(f"不支持的激活函数: {self.activation}")
    
    def forward(self, X):
        """
        前向传播
        参数:
            X: 输入矩阵，形状为(n_samples, n_features)
        返回:
            output: 输出，形状为(n_samples,)
            z: 净输入，形状为(n_samples,)
        """
        # 计算净输入：z = w^T * x + b
        z = np.dot(X, self.weights) + self.bias
        # 应用激活函数
        output = self.activation_func(z)
        return output, z
    
    def backward(self, X, z, output, grad_output):
        """
        反向传播（计算梯度）
        """
        # 计算激活函数的导数
        activation_derivative = self.activation_derivative(z)
        # 链式法则：∂L/∂z = ∂L/∂output * ∂output/∂z
        grad_z = grad_output * activation_derivative
        # 计算权重和偏置梯度
        grad_weights = np.mean(grad_z[:, np.newaxis] * X, axis=0)
        grad_bias = np.mean(grad_z)
        return grad_weights, grad_bias
    
    def update(self, grad_weights, grad_bias, learning_rate=0.01):
        """更新权重和偏置"""
        self.weights -= learning_rate * grad_weights
        self.bias -= learning_rate * grad_bias

print("神经元类定义完成！")


## 4. 总结与思考

### 关键知识点总结

1. **激活函数的作用**：
   - 引入非线性，使神经网络能够学习复杂模式
   - 控制输出范围
   - 影响梯度传播

2. **常用激活函数**：
   - **Sigmoid**：输出[0,1]，适合概率输出，但容易梯度消失
   - **Tanh**：输出[-1,1]，零中心化，比Sigmoid梯度更大
   - **ReLU**：最常用，计算简单，解决梯度消失，但可能Dead ReLU
   - **Leaky ReLU**：ReLU的改进，解决Dead ReLU问题

3. **选择原则**：
   - 隐藏层：通常使用ReLU或Leaky ReLU
   - 输出层：二分类用Sigmoid，多分类用Softmax

4. **神经元与感知器的区别**：
   - 感知器使用阶跃函数（不可微）
   - 神经元使用可微激活函数，可以使用梯度下降训练

### 思考问题

1. 为什么ReLU在隐藏层中比Sigmoid更常用？
2. 梯度消失问题是如何产生的？如何解决？
3. 为什么Tanh比Sigmoid更适合隐藏层？

### 下一步学习

- 学习如何组合多个神经元（网络结构）
- 学习前向传播算法（多层网络）
- 学习反向传播算法（如何训练多层网络）

---

**恭喜！你已经掌握了神经元的基本原理和实现！** 🎉
